In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [10]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    #sentences.append(review_to_sentences(review, tokenizer))
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    #sentences.append(review_to_sentences(review, tokenizer))
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set
Parsing sentences from unlabeled set


In [11]:
print len(sentences)

795538


In [12]:
print sentences[0]

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [13]:
print sentences[1]

[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
    
Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [14]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-03-27 22:37:15,704 : INFO : collecting all words and their counts
2017-03-27 22:37:15,705 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-03-27 22:37:15,752 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-03-27 22:37:15,790 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2017-03-27 22:37:15,823 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2017-03-27 22:37:15,859 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2017-03-27 22:37:15,890 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-03-27 22:37:15,924 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-03-27 22:37:15,960 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-03-27 22:37:15,994 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-03-27 22:37:16,030 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 4813

In [15]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [16]:
model.doesnt_match("france england germany berlin".split())
'berlin'

'berlin'

In [17]:
model.doesnt_match("paris berlin london austria".split())

'paris'

In [18]:
model.most_similar("man")

[(u'woman', 0.620313286781311),
 (u'lad', 0.5934368371963501),
 (u'lady', 0.5863518714904785),
 (u'monk', 0.5385408997535706),
 (u'chap', 0.5358078479766846),
 (u'guy', 0.5134960412979126),
 (u'person', 0.5052042603492737),
 (u'soldier', 0.5010371208190918),
 (u'men', 0.4988691508769989),
 (u'farmer', 0.4971473515033722)]

In [19]:
model.most_similar("queen")

[(u'princess', 0.6646560430526733),
 (u'bride', 0.6435585021972656),
 (u'belle', 0.6041259765625),
 (u'showgirl', 0.6019414067268372),
 (u'maid', 0.5876890420913696),
 (u'victoria', 0.5791400671005249),
 (u'eva', 0.5774714350700378),
 (u'regina', 0.5707652568817139),
 (u'stepmother', 0.5659473538398743),
 (u'vixen', 0.5655236840248108)]

In [20]:
model.most_similar("awful")

[(u'terrible', 0.7786831259727478),
 (u'atrocious', 0.7427459359169006),
 (u'horrible', 0.7425686717033386),
 (u'abysmal', 0.7124002575874329),
 (u'dreadful', 0.7117214202880859),
 (u'horrendous', 0.6795046329498291),
 (u'appalling', 0.6662834882736206),
 (u'horrid', 0.6597735285758972),
 (u'lousy', 0.6544453501701355),
 (u'amateurish', 0.6276208162307739)]

In [23]:
model.most_similar("head"), model.doesnt_match("hea")

([(u'nose', 0.6946029663085938),
  (u'chest', 0.6820085644721985),
  (u'neck', 0.6798266768455505),
  (u'fingers', 0.665209949016571),
  (u'throat', 0.6602528095245361),
  (u'finger', 0.6588197946548462),
  (u'shoulder', 0.6495363116264343),
  (u'leg', 0.6470134258270264),
  (u'pants', 0.6278905272483826),
  (u'teeth', 0.626389741897583)],
 [(u'glossed', 0.5493491291999817),
  (u'emoting', 0.47131386399269104),
  (u'dramatized', 0.4577822983264923),
  (u'gloss', 0.4372343420982361),
  (u'hyped', 0.4270886182785034),
  (u'drool', 0.42291659116744995),
  (u'sized', 0.4228958785533905),
  (u'cuckoo', 0.41692614555358887),
  (u'tip', 0.405207097530365),
  (u'edge', 0.40096810460090637)])